# 2. 載入預訓練模型與資料集

在本筆記本中，我們將學習如何從 Hugging Face Hub 載入一個預訓練的大型語言模型 (LLM) 及其對應的 Tokenizer。同時，我們也會載入一個用於後續微調任務的資料集。

## 步驟 1: 匯入必要的函式庫

首先，我們從 `transformers` 和 `torch` 匯入本次操作需要的模組。

- `AutoTokenizer`: 一個方便的類別，可以自動根據模型名稱實例化對應的 Tokenizer。
- `AutoModelForCausalLM`: 一個用於載入自回歸語言模型（如 Llama-2）的類別。
- `BitsAndBytesConfig`: 用於設定模型量化，這是實現 QLoRa 的關鍵。
- `torch`: PyTorch 函式庫。


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


## 步驟 2: 設定模型與 Tokenizer

我們將使用 Meta 開源的 `Llama-2-7b-chat-hf` 模型作為我們的基礎模型。

為了在記憶體有限的環境（如單張消費級 GPU）中載入這個 70 億參數的模型，我們將採用 **4-bit 量化**技術。`BitsAndBytesConfig` 讓我們可以精確地設定量化細節：

- `load_in_4bit=True`: 啟用 4-bit 量化。
- `bnb_4bit_quant_type="nf4"`: 使用 NF4 (NormalFloat 4) 量化類型，這是一種專為常態分佈權重設計的先進量化方法。
- `bnb_4bit_compute_dtype=torch.bfloat16`: 在計算過程中（如矩陣乘法），使用 bfloat16 類型以保持精度和穩定性。

**注意**: 第一次下載模型權重時，會需要一些時間。


In [ ]:
from huggingface_hub import login
import os
import getpass

# 建議使用環境變數或 getpass 來管理您的 Hugging Face Token，避免將密鑰直接寫在程式碼中。
# 方式一：從環境變數讀取 (推薦用於自動化環境)
# hf_token = os.environ.get("HF_TOKEN")

# 方式二：使用 getpass 在執行時手動輸入 (推薦用於互動式環境)
# 如果 token 尚未定義，則提示使用者輸入
if "hf_token" not in locals() or not hf_token:
    try:
        # 在 .ipynb 中，getpass 可能無法正常運作，這裡提供一個備用方案
        hf_token = getpass.getpass("請貼上您的 Hugging Face Access Token: ")
    except Exception:
        print("無法使用 getpass。請手動在下方設定您的 token。")
        # 如果 getpass 失敗（例如在某些非互動式環境中），可以在此處手動貼上 token
        hf_token = "YOUR_HF_TOKEN_HERE" 

# 登入 Hugging Face
if hf_token and hf_token != "YOUR_HF_TOKEN_HERE":
    login(token=hf_token)
    print("✅ Hugging Face 登入成功")
else:
    print("⚠️ 未提供 Hugging Face token。請執行此儲存格並手動輸入，或將 'YOUR_HF_TOKEN_HERE' 替換為您的 token。")


✅ Hugging Face 登入成功


In [4]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

# 設定量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 載入 Tokenizer
# 注意：你需要先到 Hugging Face 網站申請 Llama-2 的使用權限，並使用 `huggingface-cli login` 登入
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token # 設定 pad_token

# 載入量化後的模型
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto", # 自動將模型分配到可用硬體
)

print("模型與 Tokenizer 載入成功！")
print(model)




tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

模型與 Tokenizer 載入成功！
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )


## 步驟 3: 載入資料集

現在模型已經準備好，我們需要一個資料集來進行微調。這裡我們使用 `mlabonne/guanaco-llama2-1k`，這是一個已經格式化好的、適合 Llama-2 指令微調的小型資料集。

我們使用 `datasets` 函式庫中的 `load_dataset` 來載入它。


In [5]:
from datasets import load_dataset

dataset_name = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(dataset_name, split="train")

print("資料集載入成功！")
print(dataset)
print("範例資料：")
print(dataset[0])


資料集載入成功！
Dataset({
    features: ['text'],
    num_rows: 1000
})
範例資料：
{'text': '<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>'}



模型和資料都已備妥！在下一個筆記本 `03-apply-lora.ipynb` 中，我們將正式開始進行 LoRA 微調。
